In [1]:
import pandas as pd
import numpy as np

# 개요

경매데이터에 유동인구수를 변수로 추가하고자 한다.

## 계획

1. 사용할 데이터와 변수 탐색 <br>
1) 경매 데이터<br>
: 우편번호로 최종 merge버전과 merge<br>
2) 상권코드 데이터<br>
: 도로명, 유동인구<br>
3) 생활인구 데이터<br>
: 행정동코드(법정동데이터로 변환), 생활인구<br>
4) 우편번호 데이터<br>
: 우편번호, 도로명, 법정동코드<br>
5) 상권추정매출 데이터<br>
: 상권_코드_명, 당월_매출_금액, 당월_매출_건수<br>

2. merge 단계<br>
1) merge_1<br>
: 우편번호 데이터와 상권코드 데이터 merge<br>
: key = 도로명<br>
2) merge_2<br>
: merge_1 데이터와 생활인구데이터 merge<br>
: key = 법정동코드<br>
3) merge_3<br>
: merge_2데이터와 경매 데이터<br>
: key = 우편번호<br>

# 데이터 읽어오기

## 경매 크롤링

In [2]:
# 경매 크롤링
auction = pd.read_csv('1_상가경매물건_1.csv', header='infer', engine='python', encoding='utf-8-sig', sep=',')

In [12]:
auction.columns

Index(['사건번호', '경매구분', '매각기일', '주소', '도로명주소', '우편번호', '용도', '대지권', '전용면적',
       '매각대상', '사건접수날짜', '감정가', '최저가', '청구금액', '개시결정', '기일현황', '물건현황_토지이용계획',
       '감정평가현황', '임차인_대항력여부', '등기사항_소멸여부', '상세페이지링크'],
      dtype='object')

In [11]:
auction.head()

,사건번호,경매구분,매각기일,주소,도로명주소,우편번호,용도,대지권,전용면적,매각대상,...,감정가,최저가,청구금액,개시결정,기일현황,물건현황_토지이용계획,감정평가현황,임차인_대항력여부,등기사항_소멸여부,상세페이지링크
0,2014 타경 22407 (임의),임의,2015-03-10,서울특별시 중구 충무로2가 63-5 충무로패션몰리지상가 제3층 제338호,(04537) 서울특별시 중구 충무로2가 63-5 충무로패션몰리지상가 제3층 제33...,04537,상가(점포),0.66㎡ (0.2평),2.38㎡ (0.72평),토지/건물일괄매각,...,"20,000,000원","(80%) 16,000,000원","300,000,000원",2014-08-19,"회차 매각기일 최저매각금액 결과\n신건 2015-02-03 20,000,000원 유...","지하철 4호선 명동역 북측 인근에 위치\n북동측 인근으로 명동성당, 계성여자고교, ...","감정평가현황 명문감정\n가격시점 2014-09-11\n감정가 20,000,000원 ...",배당종기일: 2014-10-30\n남OO 없음\n사업 : 없음\n확정 : 없음\n배...,"소유권 이전\n 2007-07-18 집합\n 최O\n (거래가) 10,000,...",http://www.speedauction.co.kr/v3/M_view/info_r...
1,2014 타경 22407 (임의),임의,2015-03-10,서울특별시 중구 충무로2가 63-5 충무로패션몰리지상가 제3층 제340호,(04537) 서울특별시 중구 충무로2가 63-5 충무로패션몰리지상가 제3층 제34...,04537,상가(점포),0.66㎡ (0.2평),2.47㎡ (0.75평),토지/건물일괄매각,...,"21,000,000원","(80%) 16,800,000원","300,000,000원",2014-08-19,"회차 매각기일 최저매각금액 결과\n신건 2015-02-03 21,000,000원 유...","지하철 4호선 명동역 북측 인근에 위치\n북동측 인근으로 명동성당, 계성여자고교, ...","감정평가현황 명문감정\n가격시점 2014-09-11\n감정가 21,000,000원 ...",배당종기일: 2014-10-30\n남OO 없음\n사업 : 없음\n확정 : 없음\n배...,소유권 이전\n 2000-10-26 집합\n 장OO\n매매\n소유권 이전\n ...,http://www.speedauction.co.kr/v3/M_view/info_r...
2,2014 타경 22407 (임의),임의,2015-03-10,서울특별시 중구 충무로2가 63-5 충무로패션몰리지상가 제3층 제362호,(04537) 서울특별시 중구 충무로2가 63-5 충무로패션몰리지상가 제3층 제36...,04537,상가(점포),0.73㎡ (0.22평),2.92㎡ (0.88평),토지/건물일괄매각,...,"25,000,000원","(80%) 20,000,000원","300,000,000원",2014-08-19,"회차 매각기일 최저매각금액 결과\n신건 2015-02-03 25,000,000원 유...","지하철 4호선 명동역 북측 인근에 위치\n북동측 인근으로 명동성당, 계성여자고교, ...","감정평가현황 명문감정\n가격시점 2014-09-11\n감정가 25,000,000원 ...",배당종기일: 2014-10-30\n남OO 없음\n사업 : 없음\n확정 : 없음\n배...,소유권 이전\n 2000-10-26 집합\n 김OO\n매매\n소유권 이전\n ...,http://www.speedauction.co.kr/v3/M_view/info_r...
3,2014 타경 5200 (임의),임의,2015-03-11,서울특별시 양천구 신월동 992-4 뉴쇼핑프라자 지하층 1호,(08041) 서울특별시 양천구 신월동 992-4 뉴쇼핑프라자 지하층 1호\n[도로...,08041,상가(점포),36.485㎡ (11.04평),77.43㎡ (23.42평),토지/건물지분매각,...,"114,000,000원","(26%) 29,884,000원","300,904,109원",2014-03-07,"회차 매각기일 최저매각금액 결과\n신건 2014-07-23 114,000,000원 ...",강월초등교 북동측인근에 위치\n부근은 왕북 2차선변 근린생활시설및 주상복합등으로 형...,"감정평가현황 CS감정\n가격시점 2014-03-17\n감정가 114,000,000원...",배당종기일: 2014-05-20\n심OO 없음\n사업 : 없음\n확정 : 없음\n배...,소유권 이전\n 1988-10-07 집합\n 김OOOOO\n보존\n(근)저당(지...,http://www.speedauction.co.kr/v3/M_view/info_r...
4,2011 타경 7940 (임의),임의,2015-03-12,서울특별시 성북구 종암동 132 종암우림카이져팰리스 1층 118호,(02797) 서울특별시 성북구 종암동 132 종암우림카이져팰리스 1층 118호\n...,02797,상가(점포),대지권미등기,54.575㎡ (16.51평),토지/건물지분매각,...,"583,000,000원","(26%) 152,830,000원","258,806,755원",2011-03-21,"회차 매각기일 최저매각금액 결과\n신건 2014-06-26 583,000,000원 ...","종암경찰서 동측 인근에 위치\n주위는 중.소규모점포 및 근린생활시설, 상업용빌딩, ...","감정평가현황 남일감정\n가격시점 2011-03-29\n감정가 583,000,000원...",배당종기일: 2011-06-13\n김OO 없음\n사업 : 없음\n확정 : 없음\n배...,소유권(지분) 이전\n 2008-06-27 집합\n 오OOOOO\n보존\n(근)...,http://www.speedauction.co.kr/v3/M_view/info_r...


## 상권코드 데이터

In [3]:
# 상권 코드
building_code = pd.read_csv('2_building_code.csv', engine='python', header='infer', encoding = 'euc-kr')

In [13]:
building_code.columns

Index(['기준_년월_코드', '상권_코드', '상권_코드_명', '총_유동인구_수'], dtype='object')

In [4]:
building_code.columns = ['기준년월', '상권코드', '도로명', '유동인구']

In [5]:
building_code.head()

,기준년월,상권코드,도로명,유동인구
0,201807,1,가락로16길,12945
1,201807,2,가락로28길,37883
2,201807,3,가락로36길,25330
3,201807,4,가로공원로76길,53739
4,201807,5,가마산로12길,33914


In [6]:
building_code = building_code.drop(['기준년월', '상권코드'], axis = 1)

### 사용할 데이터 보기

In [32]:
building_code.head()

,도로명,유동인구
0,가락로16길,12945
1,가락로28길,37883
2,가락로36길,25330
3,가로공원로76길,53739
4,가마산로12길,33914


In [64]:
building_code.shape

(6940, 2)

## 생활인구 데이터

In [7]:
# 생활인구
population = pd.read_csv('4_생활인구.csv', engine='python', header='infer', encoding='UTF-8')

In [18]:
population.columns

Index(['기준일ID', '시간대구분', '행정동코드', '총생활인구수'], dtype='object')

In [19]:
population.head()

,기준일ID,시간대구분,행정동코드,총생활인구수
0,20180701,0,11215760,19006.4504
1,20180701,0,11710632,30437.2743
2,20180701,0,11590630,38560.4198
3,20180701,0,11590660,43559.7821
4,20180701,0,11590650,16676.7655


In [8]:
population = population.drop(['기준일ID', '시간대구분'], axis=1)

In [9]:
population_gruop = population.groupby("행정동코드").mean()
population_gruop.reset_index()

,행정동코드,총생활인구수
0,11110515,16551.054077
1,11110530,31276.468667
2,11110540,6941.842364
3,11110550,12913.703762
4,11110560,19744.041674
5,11110570,7243.537884
6,11110580,8771.958811
7,11110600,8088.628950
8,11110615,71579.869659
9,11110630,20416.298964


In [218]:
population_gruop.shape

(424, 1)

In [10]:
# 법정동-행정동 코드
dong_mapping = pd.read_csv('3_code.csv', engine='python', header='infer', encoding='euc-kr')

In [11]:
dong_mapping['행정동코드'] = dong_mapping['행정동코드']//100

In [39]:
dong_mapping.head()

,행정동코드,법정동코드
0,11110515,1111010100
1,11110515,1111010200
2,11110515,1111010300
3,11110515,1111010400
4,11110515,1111010500


In [41]:
dong_mapping.shape

(741, 2)

### 버전 1

In [12]:
# 생활인구 + 법정동행정동 merge
population_code_chage = pd.merge(population_gruop, dong_mapping, how = 'left', on = '행정동코드')

In [47]:
population_code_chage.shape

(741, 3)

In [49]:
population_code_chage.tail()

,행정동코드,총생활인구수,법정동코드
736,11740650,28492.304160,1174010800
737,11740660,29312.568610,1174010800
738,11740685,55445.597574,1174010500
739,11740690,7758.858306,1174010600
740,11740700,29446.507401,1174010600


In [13]:
population_code_chage_1 = population_code_chage.drop(['행정동코드'], axis=1)

In [14]:
population_code_chage_2 = population_code_chage_1.groupby("법정동코드").mean()

In [53]:
population_code_chage_2.head()

,총생활인구수
법정동코드,
1111010100,16551.054077
1111010200,16551.054077
1111010300,16551.054077
1111010400,16551.054077
1111010500,16551.054077


In [92]:
population_code_chage.tail()

,행정동코드,총생활인구수,법정동코드
736,11740650,28492.304160,1174010800
737,11740660,29312.568610,1174010800
738,11740685,55445.597574,1174010500
739,11740690,7758.858306,1174010600
740,11740700,29446.507401,1174010600


==> 버전1의 오류: 원데이터에서는 행정동이 큰 범위였는데 merge후에보니 법정동코드가 더 큰 범위

### 버전2

In [15]:
# 생활인구 + 법정동행정동 merge
pop_code_change = pd.merge(dong_mapping, population_gruop, how = 'left', on = '행정동코드')

#### 사용할 데이터 보기

In [222]:
print(pop_code_change.shape)
pop_code_change.head(10)

(741, 3)


,행정동코드,법정동코드,총생활인구수
0,11110515,1111010100,16551.054077
1,11110515,1111010200,16551.054077
2,11110515,1111010300,16551.054077
3,11110515,1111010400,16551.054077
4,11110515,1111010500,16551.054077
5,11110515,1111010800,16551.054077
6,11110515,1111010900,16551.054077
7,11110515,1111011000,16551.054077
8,11110515,1111011100,16551.054077
9,11110515,1111011900,16551.054077


## 우편번호 데이터

In [16]:
# 우편번호 DB
raw_addr_code = pd.read_csv('서울특별시.csv', names = ['우편번호','시도','시도영문','시군구','시군구영문','읍면','읍면영문','도로명코드','도로명','도로명영문','지하여부','건물번호본번','건물번호부번','건물관리번호','다량배달처명','시군구용건물명','법정동코드','법정동명','리명','행정동명','산여부','지번본번','읍면동일련번호','지번부번','구우편번호','우편번호일련번호'],sep='|', engine='python')

우편번호 DB 1차 정리

In [9]:
raw_addr_code

,우편번호,시도,시군구,도로명코드,도로명,지하여부,건물번호본번,건물번호부번,건물관리번호,법정동코드,법정동명,행정동명,지번본번,읍면동일련번호,지번부번
0,06314,서울특별시,강남구,1.168042e+11,논현로6길,0.0,18.0,0.0,1168010300101710000021134,1168010300,개포동,개포4동,1171.0,1.0,0.0
1,06314,서울특별시,강남구,1.168042e+11,논현로6길,0.0,30.0,5.0,1168010300101750008021066,1168010300,개포동,개포4동,1175.0,1.0,8.0
2,06314,서울특별시,강남구,1.168042e+11,논현로6길,0.0,22.0,4.0,1168010300101720001021121,1168010300,개포동,개포4동,1172.0,1.0,1.0
3,06314,서울특별시,강남구,1.168042e+11,논현로6길,0.0,26.0,2.0,1168010300101730001021090,1168010300,개포동,개포4동,1173.0,1.0,1.0
4,06314,서울특별시,강남구,1.168042e+11,논현로6길,0.0,19.0,0.0,1168010300101890007021099,1168010300,개포동,개포4동,1189.0,1.0,7.0
5,06314,서울특별시,강남구,1.168042e+11,논현로6길,0.0,36.0,9.0,1168010300101760004028175,1168010300,개포동,개포4동,1176.0,1.0,4.0
6,06303,서울특별시,강남구,1.168042e+11,개포로15길,0.0,32.0,9.0,1168010300102430005020337,1168010300,개포동,개포4동,1243.0,1.0,5.0
7,06303,서울특별시,강남구,1.168042e+11,개포로15길,0.0,13.0,0.0,1168010300102400000020392,1168010300,개포동,개포4동,1240.0,1.0,0.0
8,06303,서울특별시,강남구,1.168042e+11,개포로15길,0.0,5.0,0.0,1168010300102400004020476,1168010300,개포동,개포4동,1240.0,1.0,4.0
9,06303,서울특별시,강남구,1.168042e+11,개포로15길,0.0,3.0,13.0,1168010300102390014000001,1168010300,개포동,개포4동,1239.0,1.0,14.0


In [162]:
raw_addr_code.columns

Index(['우편번호', '시도', '시군구', '도로명코드', '도로명', '지하여부', '건물번호본번', '건물번호부번',
       '건물관리번호', '법정동코드', '법정동명', '행정동명', '지번본번', '읍면동일련번호', '지번부번'],
      dtype='object')

In [243]:
raw_addr_code.isnull().sum()

우편번호           0
시도          1160
시군구         1160
도로명코드       1160
도로명         1160
지하여부        1160
건물번호본번      1160
건물번호부번      1160
건물관리번호      1160
법정동코드       1160
법정동명        1165
행정동명       31683
지번본번        1160
읍면동일련번호     1160
지번부번        1165
dtype: int64

In [17]:
raw_addr_code = raw_addr_code.drop(['시도영문', '시군구영문', '읍면영문', '도로명영문','리명', '산여부','읍면', '다량배달처명','시군구용건물명', '구우편번호', '우편번호일련번호'], axis=1)

In [20]:
raw_addr_code.columns

Index(['우편번호', '시도', '시군구', '도로명코드', '도로명', '지하여부', '건물번호본번', '건물번호부번',
       '건물관리번호', '법정동코드', '법정동명', '행정동명', '지번본번', '읍면동일련번호', '지번부번'],
      dtype='object')

In [18]:
# addr_code_1 : 우편번호, 도로명코드, 법정동코드
addr_code_1 = raw_addr_code.drop(['시도', '시군구', '도로명코드', '지하여부', '건물번호본번', '건물번호부번',
       '건물관리번호', '법정동명', '행정동명', '지번본번', '읍면동일련번호', '지번부번'], axis=1)

In [63]:
addr_code_1.head()

,우편번호,도로명,법정동코드
0,06314,논현로6길,1168010300
1,06314,논현로6길,1168010300
2,06314,논현로6길,1168010300
3,06314,논현로6길,1168010300
4,06314,논현로6길,1168010300


In [246]:
addr_code_1.shape

(568028, 3)

In [66]:
# 동일한 관측치행 지우기
addr_code_2 = addr_code_1.drop_duplicates()

In [67]:
addr_code_2.shape

(33301, 3)

### 사용할 데이터보기

In [69]:
addr_code_2.head()

,우편번호,도로명,법정동코드
0,06314,논현로6길,1168010300
6,06303,개포로15길,1168010300
14,06314,논현로4길,1168010300
16,06315,논현로4길,1168010300
24,06322,언주로6길,1168010300


## 상권추정매출

In [19]:
sales = pd.read_csv('5_상권추정매출.csv', engine='python', header='infer', encoding = 'euc-kr')

In [171]:
sales.head()

,상권_코드_명,당월_매출_금액,당월_매출_건수
0,희우정로16길,1748879,13
1,희우정로16길,1529585,201
2,희우정로16길,4100714,478
3,희우정로16길,5373213,371
4,희우정로16길,39095254,282


In [20]:
sales.columns = ['도로명', '매출금액', '매출건수']

In [21]:
sales_group = sales.groupby(['도로명']).mean()
print(sales_group.shape)
sales_group.reset_index().head()

(1738, 2)


,도로명,매출금액,매출건수
0,가락로16길,6.762326e+07,2827.043478
1,가락로28길,4.173171e+07,2385.160000
2,가락로36길,3.134447e+07,1556.727273
3,가로공원로76길,1.104793e+08,5061.108108
4,가마산로12길,4.259197e+06,179.500000


# 데이터 확인

In [ ]:
print()

# 데이터 맵핑

## 맵핑버전1

### merge_0 = 상권코드 + 상권매출

In [184]:
merge_0 = pd.merge(building_code, sales_group, how = 'left', on = '도로명')

In [185]:
print(merge_0.shape)
print(building_code.shape)
print(sales_group.shape)

(6940, 4)
(6940, 2)
(1738, 2)


In [186]:
merge_0.head()

,도로명,유동인구,매출금액,매출건수
0,가락로16길,12945,6.762326e+07,2827.043478
1,가락로28길,37883,4.173171e+07,2385.160000
2,가락로36길,25330,3.134447e+07,1556.727273
3,가로공원로76길,53739,1.104793e+08,5061.108108
4,가마산로12길,33914,4.259197e+06,179.500000


### merge_1 =  merge_0 + 우편번호

In [187]:
merge_1 = pd.merge(merge_0, addr_code_2, how = 'left', on = '도로명')

In [188]:
merge_1.head()

,도로명,유동인구,매출금액,매출건수,우편번호,법정동코드
0,가락로16길,12945,6.762326e+07,2827.043478,05696,1171010500
1,가락로16길,12945,6.762326e+07,2827.043478,05697,1171010500
2,가락로28길,37883,4.173171e+07,2385.160000,05671,1171010400
3,가락로36길,25330,3.134447e+07,1556.727273,05634,1171011100
4,가락로36길,25330,3.134447e+07,1556.727273,05644,1171011100


In [189]:
merge_1.shape

(16884, 6)

In [190]:
merge_1 = merge_1.drop(['도로명'], axis=1)

In [191]:
merge_1.head()

,유동인구,매출금액,매출건수,우편번호,법정동코드
0,12945,6.762326e+07,2827.043478,05696,1171010500
1,12945,6.762326e+07,2827.043478,05697,1171010500
2,37883,4.173171e+07,2385.160000,05671,1171010400
3,25330,3.134447e+07,1556.727273,05634,1171011100
4,25330,3.134447e+07,1556.727273,05644,1171011100


In [192]:
merge_1_check = merge_1.groupby(['법정동코드', '우편번호']).mean()

In [193]:
merge_1_check.reset_index().head()

,법정동코드,우편번호,유동인구,매출금액,매출건수
0,1111010100,03031,1536.75,29343888.85,947.4
1,1111010100,03046,1244.00,7959872.00,1145.5
2,1111010100,03047,1244.00,7959872.00,1145.5
3,1111010200,03031,1829.50,50727905.70,749.3
4,1111010200,03032,1829.50,50727905.70,749.3


==> 해석

법정동코드가 우편번호보다 큰단위여서 생기는일

생활인구까지 merge한 이후 법정동코드는 생략하자

### merge_2  = merge_1 + 생활인구

In [223]:
print(type(merge_1['법정동코드'][1]))
print(type(pop_code_change['법정동코드'][1]))

<class 'str'>
<class 'numpy.int64'>


In [224]:
pop_code_change['법정동코드'] = pop_code_change['법정동코드'].astype('str')

In [225]:
merge_2 = pd.merge(merge_1, pop_code_change, how = 'left', on = '법정동코드')

In [226]:
merge_2.head()

,유동인구,매출금액,매출건수,우편번호,법정동코드,행정동코드,총생활인구수
0,12945,6.762326e+07,2827.043478,05696,1171010500,11710600.0,29881.930533
1,12945,6.762326e+07,2827.043478,05697,1171010500,11710600.0,29881.930533
2,37883,4.173171e+07,2385.160000,05671,1171010400,11710580.0,28851.660166
3,37883,4.173171e+07,2385.160000,05671,1171010400,11710590.0,18494.718389
4,25330,3.134447e+07,1556.727273,05634,1171011100,11710561.0,16895.417600


#### 버전1

In [198]:
merge_2_1 = merge_2.drop(['법정동코드', '행정동코드'], axis = 1)

In [199]:
merge_2_1.head(20)

,유동인구,매출금액,매출건수,우편번호,총생활인구수
0,12945,6.762326e+07,2827.043478,05696,29881.930533
1,12945,6.762326e+07,2827.043478,05697,29881.930533
2,37883,4.173171e+07,2385.160000,05671,28851.660166
3,37883,4.173171e+07,2385.160000,05671,18494.718389
4,25330,3.134447e+07,1556.727273,05634,16895.417600
5,25330,3.134447e+07,1556.727273,05634,35372.228146
6,25330,3.134447e+07,1556.727273,05634,22367.755086
7,25330,3.134447e+07,1556.727273,05644,16895.417600
8,25330,3.134447e+07,1556.727273,05644,35372.228146
9,25330,3.134447e+07,1556.727273,05644,22367.755086


==> 해야할 일: 유동인구와 우편번호가 같으면 묶어서 한 행을 만들고 그때 총생활인구수는 평균을 내자

In [200]:
merge_2_1['유동인구'] = merge_2_1['유동인구'].astype('str')
merge_2_1['우편번호'] = merge_2_1['우편번호'].astype('str')

In [201]:
a = merge_2_1.groupby(['유동인구', '우편번호']).mean()

In [202]:
a= a.reset_index()
a

,유동인구,우편번호,매출금액,매출건수,총생활인구수
0,100,02401,NaN,NaN,23888.493500
1,100017,04576,4.109385e+07,1207.208333,13403.602334
2,100040,02496,6.481472e+07,5170.843750,19274.809766
3,100040,02498,6.481472e+07,5170.843750,19274.809766
4,100040,02499,6.481472e+07,5170.843750,19274.809766
5,10011,04377,1.239159e+08,9810.750000,49829.539383
6,10011,04378,1.239159e+08,9810.750000,49829.539383
7,100162,08785,8.615603e+07,5172.000000,24216.244501
8,10018,02764,3.236338e+07,976.727273,13775.306322
9,10018,02765,3.236338e+07,976.727273,13775.306322


#### 버전 2

In [227]:
merge_2_2 = merge_2.drop(['법정동코드', '행정동코드'], axis = 1)

In [228]:
merge_2_2['우편번호'] = merge_2_2['우편번호'].astype('str')

In [229]:
b = merge_2_2.groupby('우편번호').mean()
b= b.reset_index()
b

,우편번호,유동인구,매출금액,매출건수,총생활인구수
0,01000,129.250000,8.525115e+07,1015.400000,18951.356447
1,01002,6818.250000,5.603055e+07,3053.500000,18951.356447
2,01003,19503.750000,2.134473e+07,1118.107609,18951.356447
3,01004,5061.500000,1.900404e+07,1217.650000,18951.356447
4,01005,19503.750000,2.134473e+07,1118.107609,18951.356447
5,01009,22366.500000,3.957771e+07,2666.370370,23871.007881
6,01010,22366.500000,3.957771e+07,2666.370370,23051.065975
7,01011,22366.500000,3.957771e+07,2666.370370,23871.007881
8,01014,25982.625000,1.957295e+07,1151.810294,23871.007881
9,01015,25982.625000,1.957295e+07,1151.810294,23871.007881


In [203]:
a.isnull().sum()

유동인구       0
우편번호       0
매출금액      20
매출건수      20
총생활인구수    16
dtype: int64

In [206]:
type(a.우편번호[1])

str

데이터저장

In [230]:
b.to_csv('매출_생활+유동인구_맵핑_b.csv', header='infer', encoding='utf-8-sig', sep=',',index=False)

### 경매데이터 + merge_2(최종 데이터프레임 : a)

미완성

In [153]:
auction_plus = pd.read_csv('1_경매_데이터.csv', header='infer', engine='python', encoding='utf-8-sig', sep=',')

In [154]:
auction_plus.shape

(3188, 44)

In [161]:
auction_plus.head()

,사건번호_x,중복사건여부,매각기일_x,주소_x,대지(토지)면적,전용(건물)면적,매각대상_x,유치권여부,대항력있는임차인여부,위반건축물여부,...,감정가_y,최저가_y,청구금액,개시결정,기일현황,물건현황_토지이용계획,감정평가현황,임차인_대항력여부,등기사항_소멸여부,상세페이지링크
0,서울동부 2011-9840,N,2015-01-05,서울특별시 광진구 구의동 546-4 테크노-마트21 6층 디065호,NaN,3.4평,상가(점포),N,N,N,...,"120,000,000원","(5%) 6,597,000원","88,957,501원",2011-07-13,"회차 매각기일 최저매각금액 결과\n신건 2013-05-20 120,000,000원 ...","전철 2호선 강변역 동측 인근에 위치\n부근은 대형 판매시설 및 업무시설, ,터미널...","감정평가현황 안국감정\n가격시점\n감정가 120,000,000원 \n토지 (30...",배당종기일: 2011-09-23\n김OO 없음\n사업 : 2011-03-21\n확정...,소유권 이전\n 1999-05-18 집합\n 정OO\n협의분할에 의한 상속\n소...,http://www.speedauction.co.kr/v3/M_view/pagevi...
1,서울동부 2013-9493,N,2015-01-05,"서울특별시 광진구 구의동 546-4, 5층 비023호 (구의동,테크노-마트21) 외1건",NaN,7.2평,상가(점포),N,N,N,...,"118,000,000원","(13%) 15,838,000원","161,552,623원",2013-05-21,"회차 매각기일 최저매각금액 결과\n신건 2013-10-07 118,000,000원 ...","지하철 2호선 강변역 북동측 인근에 위치\n인근에 시외버스터미널, 관공서, 한강시민...","감정평가현황 현대기업감정\n가격시점 2013-05-31\n감정가 118,000,00...",배당종기일: 2013-08-12\n이OO 없음\n사업 : 2010-06-03\n확정...,소유권 이전\n 1998-09-26 집합\n 김O\n매매\n(근)저당 소멸기준\...,http://www.speedauction.co.kr/v3/M_view/info_r...
2,서울동부 2014-3522,N,2015-01-05,서울특별시 성동구 마장동 767-27,30.2평,NaN,상가,N,N,N,...,"406,059,600원","(64%) 259,878,000원","436,327,761원",2014-03-06,"회차 매각기일 최저매각금액 결과\n신건 2014-10-27 406,059,600원 ...","지하철 5호선 마장역 북동층 인근에 위치\n부근은 단독주택, 마장현대아파트, 마장초...","감정평가현황 구정감정\n가격시점 2014-03-21\n감정가 406,059,600원...",배당종기일: 2014-05-27\n- 매각물건명세서상 조사된\n 임차내역이 없습니다,소유권 이전\n 1994-03-30 건물/토지\n 추O\n매매\n소유권 이전\n...,http://www.speedauction.co.kr/v3/M_view/info_r...
3,서울동부 2014-5955,N,2015-01-05,서울특별시 송파구 잠실동 175-1 위너스오피스텔2 10층 1005호,NaN,10.3평,오피스텔(업무),N,N,N,...,"160,000,000원","(80%) 128,000,000원","3,422,980원",2014-04-21,"회차 매각기일 최저매각금액 결과\n신건 2014-12-01 160,000,000원 ...",지하철 2호선 신천역 남서측 인근에 위치\n주변은 근린생활시설 및 업무시설 등 상업...,"감정평가현황 김일수감정\n가격시점 2014-05-02\n감정가 160,000,000...",배당종기일: 2014-07-14\n김OO 없음\n전입 : 없음\n확정 : 없음\n배...,소유권 이전\n 1991-03-14 집합\n 전O\n매매\n소유권 이전\n 2...,http://www.speedauction.co.kr/v3/M_view/info_r...
4,서울남부 2012-15814 물번[4],Y,2015-01-07,서울특별시 구로구 오류동 54-5 5층 501호,NaN,66.9평,상가(점포),Y,N,N,...,"570,000,000원","(51%) 291,840,000원","1,047,810,385원",2012-06-13,"회차 매각기일 최저매각금액 결과\n신건 2013-08-14 570,000,000원 ...",지하철 1호선 오류역 북측 인근에 위치\n부근은 주상복합건물과 근린생활시설 및 주택...,"감정평가현황 이룸감정\n가격시점 2012-06-19\n감정가 570,000,000원...",배당종기일: 2012-08-20\n염OO 없음\n사업 : 2011-03-11\n확정...,소유권 이전\n 2009-07-24 집합\n 고OOO\n보존\n소유권 이전\n ...,http://www.speedauction.co.kr/v3/M_view/info_r...


In [156]:
auction_plus.isnull().sum()

사건번호_x            0
중복사건여부            0
매각기일_x            0
주소_x              0
대지(토지)면적       2618
전용(건물)면적        576
매각대상_x            0
유치권여부             0
대항력있는임차인여부        0
위반건축물여부           0
대지권미등기여부          0
권리분석기타코드       1601
권리분석코드명        1601
감정가_x             0
유찰수             659
최저가_x             0
최저가율              0
매각가               3
매각가율              3
입찰수              45
경매결과              0
조회수               0
Index             0
사건번호_y            0
경매구분              0
매각기일_y            0
주소_y              0
도로명주소           165
우편번호            165
용도                0
대지권               0
전용면적             14
매각대상_y            0
사건접수날짜          538
감정가_y             0
최저가_y             0
청구금액              0
개시결정              0
기일현황              0
물건현황_토지이용계획       0
감정평가현황            0
임차인_대항력여부         0
등기사항_소멸여부         0
상세페이지링크           0
dtype: int64

In [157]:
#우편번호 결측치 행 지우기
auction['우편번호'].dropna(how='all')

0          04537
1          04537
2          04537
3          08041
4          02797
6          02149
7          05504
8          02568
9          01779
10         02591
11         07714
13         06600
14         08288
15         07744
16         05335
17         01701
18         01332
19         01604
20         04374
21         03766
22         03766
23         04565
24         04030
25         03408
26         04537
27         06367
28         07333
29         07345
30         03168
31         04557
          ...   
3015       08288
3016       04566
3017       04615
3018       06568
3019       07911
3020       02076
3021       02797
3022       01073
3023       04382
3024       07297
3025       01325
3026       02819
3027       01779
3028       01220
3029       08215
3030       08215
3031       08215
3032       08215
3033    봉천동,아이리스
3034       02618
3035       08004
3036       08004
3037       08288
3038       08288
3039       07555
3040       04564
3041       04564
3042       065

## 맵핑버전2

In [239]:
### 데이터 타입확인
# 도로명
print(type(addr_code_2.도로명[0])) # 우편번호
print(type(sales.도로명[0])) # 상권추정매출
print(type(building_code.도로명[0])) # 상권코드

# 법정동코드
print(type(addr_code_2.법정동코드[0])) # 우편번호
print(type(pop_code_change.법정동코드[0])) # 동코드+생활인구

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [238]:
pop_code_change['법정동코드'] = pop_code_change['법정동코드'].astype('str')

### mapping_1 = 우편번호 + 상권코드 (key: 도로명)

In [249]:
mapping_1 = pd.merge(addr_code_2, building_code, how = 'left', on = '도로명')
print(addr_code_2.shape)
print(addr_code_2.isnull().sum())
print('-------------------------')
print(mapping_1.shape)
print(mapping_1.isnull().sum())

(33301, 3)
우편번호        0
도로명      1160
법정동코드    1160
dtype: int64
-------------------------
(45952, 4)
우편번호         0
도로명       1160
법정동코드     1160
유동인구     29084
dtype: int64


In [250]:
mapping_1_1 = pd.merge(building_code, addr_code_2, how = 'left', on = '도로명')
print(mapping_1_1.shape)
print(mapping_1_1.isnull().sum())

(16884, 4)
도로명       0
유동인구      0
우편번호     16
법정동코드    16
dtype: int64


### mapping_2 = 우편번호 + 상권추정매출 (key: 도로명)

In [251]:
mapping_2 = pd.merge(addr_code_2, sales_group, how = 'left', on = '도로명')
print(addr_code_2.shape)
print(addr_code_2.isnull().sum())
print('-------------------------')
print(mapping_2.shape)
print(mapping_2.isnull().sum())

(33301, 3)
우편번호        0
도로명      1160
법정동코드    1160
dtype: int64
-------------------------
(33301, 5)
우편번호         0
도로명       1160
법정동코드     1160
매출금액     29077
매출건수     29077
dtype: int64


In [253]:
mapping_2_1 = pd.merge(sales_group, addr_code_2, how = 'left', on = '도로명')
print(mapping_2_1.shape)
print(mapping_2_1.isnull().sum())

(4228, 5)
도로명      0
매출금액     0
매출건수     0
우편번호     4
법정동코드    4
dtype: int64


### mapping_3 = mapping_1 + 상권추정매출  (key: 도로명)

### mapping_4 = 우편번호 + [생활인구+맵핑코드] (key: 법정동코드)

### mapping_5 = 3번 + 4번 (key: 우편번호)